### Year 3 Topology 3 Contingency Analysis Reporting - Lower Limit Violation

In [1]:
import pandas as pd
import numpy as np

#### Bus Voltage Data

In [2]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
list_gen_hydro = [400,500,600] 
for gen in list_gens:
    for lsc in list_lsc:
        for gen_hy in list_gen_hydro:
            file_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'.xlsx'
            data_volt = pd.read_excel(file_out, sheet_name='Bus Voltage')
            data_v_nonan = data_volt.dropna(how='all').reset_index(drop=True)
            data_v_nonan['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            list_volt.append(data_v_nonan)
data_v = pd.concat(list_volt).reset_index(drop=True)
print(len(data_v))
display(data_v.head())

50204


,BUS,RECORD,TYPE,MIN/DROP,MAX/RISE,CONTINGENCY,BASE VOLTS,CONT VOLTS,DEVIATION,RANGE VIO,DEV VIO,Unnamed: 11,Scenario
0,101 NUC-A 21.600,ALL,RANGE,0.95,1.05,BASE CASE,1.020000,NaN,NaN,NaN,NaN,BASE VOLTS is pu base case voltage.,"Solar = 0 MW, Hydro = 400 MW, LLS"
1,102 NUC-B 21.600,ALL,RANGE,0.95,1.05,BASE CASE,1.020000,NaN,NaN,NaN,NaN,CONT VOLTS is pu contingency case voltage.,"Solar = 0 MW, Hydro = 400 MW, LLS"
2,103 SOLAR_PV 13.800,ALL,RANGE,0.95,1.05,BASE CASE,1.000000,NaN,NaN,NaN,NaN,DEVIATION is difference between contingency ca...,"Solar = 0 MW, Hydro = 400 MW, LLS"
3,151 NUCPANT 500.00,ALL,RANGE,0.95,1.05,BASE CASE,1.014451,NaN,NaN,NaN,NaN,RANGE VIO is range violations calculated as bu...,"Solar = 0 MW, Hydro = 400 MW, LLS"
4,152 MID500 500.00,ALL,RANGE,0.95,1.05,BASE CASE,1.034070,NaN,NaN,NaN,NaN,DEV VIO is deviation violations calculated as ...,"Solar = 0 MW, Hydro = 400 MW, LLS"


#### Bus voltage data wrangling 

In [3]:
data_v = data_v.rename(columns={'BUS':'Bus', 
                                'CONTINGENCY':'Contingency',
                                'BASE VOLTS':'Base Voltage',                               
                                'CONT VOLTS':'Contingency Voltage',
                                'RANGE VIO' : 'Range Violation', 
                                'DEVIATION' : 'Deviation'})
data_v['Contingency'] = data_v['Contingency'].str.replace('&','\&')
data_v['Bus'] = data_v['Bus'].str.replace('_','\_')
data_v['Bus Number'] = data_v['Bus'].str.split(expand=True)[0]

#### Lower limit bus voltage violations 

In [4]:
data_low = data_v[data_v['Contingency Voltage']<0.9].reset_index(drop=True)
print(len(data_low))
display(data_low.head())

1090


,Bus,RECORD,TYPE,MIN/DROP,MAX/RISE,Contingency,Base Voltage,Contingency Voltage,Deviation,Range Violation,DEV VIO,Unnamed: 11,Scenario,Bus Number
0,154 DOWNTN 230.00,ALL,RANGE,0.95,1.05,SING OPN LIN 13 202-203(1),0.972030,0.877560,-0.094469,-0.072440,NaN,NaN,"Solar = 0 MW, Hydro = 400 MW, LLS",154
1,203 EAST230 230.00,ALL,RANGE,0.95,1.05,SING OPN LIN 13 202-203(1),0.992516,0.860671,-0.131844,-0.089329,NaN,NaN,"Solar = 0 MW, Hydro = 400 MW, LLS",203
2,205 SUB230 230.00,ALL,RANGE,0.95,1.05,SING OPN LIN 13 202-203(1),0.980000,0.884568,-0.095432,-0.065432,NaN,NaN,"Solar = 0 MW, Hydro = 400 MW, LLS",205
3,154 DOWNTN 230.00,LIMIT,RANGE,0.90,1.10,SING OPN LIN 13 202-203(1),0.972030,0.877560,-0.094469,-0.022440,NaN,NaN,"Solar = 0 MW, Hydro = 400 MW, LLS",154
4,203 EAST230 230.00,LIMIT,RANGE,0.90,1.10,SING OPN LIN 13 202-203(1),0.992516,0.860671,-0.131844,-0.039329,NaN,NaN,"Solar = 0 MW, Hydro = 400 MW, LLS",203


#### Reporting of Buses with lower voltage violations

In [5]:
L_scen = list(data_low['Scenario'].unique())
print(f'It was seen that for Topology 3, all the studied scenarios reported lower voltage limit violation.')
print('------------------------------------------------------------------------------------------------------------------------------------------------------')
for vlscen in L_scen:
    print(vlscen)
    # Extracting bus data with voltage less than 0.9 PU
    data_lscen = data_low[data_low['Scenario'] == vlscen]
    print(f'For the studied scenario {vlscen}, the bus(es) reporting lower emergency range violation(s) are the buses',', '.join(data_lscen['Bus Number'].unique()),)
    # lower range violation latex table data reporting
    data_replv = data_lscen[['Bus Number','Contingency','Base Voltage', 'Contingency Voltage', 'Deviation', 'Range Violation']].reset_index(drop=True)
    
    unit_lowv = data_replv[data_replv['Contingency'].str.contains('UNIT')]
    if len(unit_lowv) != 0:
        unit_lv = unit_lowv.drop_duplicates(subset=['Bus Number'])
        for contlu in list(unit_lv['Contingency'].unique()):
            unit_cont = unit_lv[unit_lv['Contingency']==contlu]
            busesu = ', '.join(unit_cont['Bus Number'].unique())
            print(f'The buses {busesu} reported violation for the unit fault {contlu}')
       
    bus_lowv = data_replv[data_replv['Contingency'].str.contains('BUS')]
    if len(bus_lowv) != 0:
        bus_lv = bus_lowv.drop_duplicates(subset=['Bus Number'])
        for contlb in list(bus_lv['Contingency'].unique()):
            bus_cont = bus_lv[bus_lv['Contingency']==contlb]
            busesb = ', '.join(bus_cont['Bus Number'].unique())
            print(f'The buses {busesb} reported violation for the bus fault {contlb}')
                    
    line_lowv = data_replv[data_replv['Contingency'].str.contains('SING OPN LIN')]
    if len(line_lowv) != 0:
        line_lv = line_lowv.drop_duplicates(subset=['Bus Number'])
        for contll in list(line_lv['Contingency'].unique()):
            line_cont = line_lv[line_lv['Contingency']==contll]
            busesl = ', '.join(line_cont['Bus Number'].unique())
            print(f'The buses {busesl} reported violation for the single line open fault {contll}')
    print('------------------------------------------------------------------------------------------------------------------------------------------------------')


It was seen that for Topology 3, all the studied scenarios reported lower voltage limit violation.
------------------------------------------------------------------------------------------------------------------------------------------------------
Solar = 0 MW, Hydro = 400 MW, LLS
For the studied scenario Solar = 0 MW, Hydro = 400 MW, LLS, the bus(es) reporting lower emergency range violation(s) are the buses 154, 203, 205
The buses 154, 203, 205 reported violation for the single line open fault SING OPN LIN   13 202-203(1)
------------------------------------------------------------------------------------------------------------------------------------------------------
Solar = 0 MW, Hydro = 500 MW, LLS
For the studied scenario Solar = 0 MW, Hydro = 500 MW, LLS, the bus(es) reporting lower emergency range violation(s) are the buses 153, 154, 203, 205, 103, 204, 3007, 3008
The buses 154, 205 reported violation for the bus fault BUS 203
The buses 153, 154 reported violation for the s

Lower Voltage Limit Counts - Scenario 

In [6]:
vl_index = list(data_low['Scenario'].value_counts().index)
vl_counts = list(data_low['Scenario'].value_counts())
dict_vl_count = {
    'Scenario':vl_index,
    'Violation Counts':vl_counts
}
scen_lv_vc  = pd.DataFrame(dict_vl_count)
scen_lv_vc

,Scenario,Violation Counts
0,"Solar = 50 MW, Hydro = 600 MW, HLS",132
1,"Solar = 100 MW, Hydro = 600 MW, HLS",102
2,"Solar = 0 MW, Hydro = 600 MW, HLS",80
3,"Solar = 0 MW, Hydro = 500 MW, HLS",76
4,"Solar = 0 MW, Hydro = 400 MW, HLS",72
5,"Solar = 50 MW, Hydro = 500 MW, HLS",52
6,"Solar = 100 MW, Hydro = 500 MW, HLS",42
7,"Solar = 100 MW, Hydro = 600 MW, LLS",42
8,"Solar = 100 MW, Hydro = 400 MW, HLS",40
9,"Solar = 0 MW, Hydro = 500 MW, RLS",40


Lower Voltage Limit Counts - Bus

In [7]:
vlb_index = list([i.strip().split()[0] for i in data_low['Bus'].value_counts().index])
vlb_counts = list(data_low['Bus'].value_counts())
dict_vlb_count = {
    'Bus':vlb_index,
    'Violation Counts':vlb_counts
}
bus_lv_vc  = pd.DataFrame(dict_vlb_count)
bus_lv_vc

,Bus,Violation Counts
0,154,308
1,205,236
2,3008,154
3,103,120
4,3007,82
5,203,74
6,204,40
7,153,30
8,3006,26
9,3018,10


Lower Voltage Limit Counts - Contingency

In [8]:
vlc_index = list(data_low['Contingency'].value_counts().index)
vlc_counts = list(data_low['Contingency'].value_counts())
dict_vlc_count = {
    'Contingency':vlc_index,
    'Violation Counts':vlc_counts
}
cont_lv_vc  = pd.DataFrame(dict_vlc_count)
cont_lv_vc

,Contingency,Violation Counts
0,SING OPN LIN 6 152-153(1),178
1,BUS 203,98
2,BUS 153,86
3,BUS 3004,82
4,SING OPN LIN 13 202-203(1),66
5,SING OPN LIN 9 201-202(1),62
6,BUS 3005,56
7,BUS 3003,50
8,SING OPN LIN 7 153-154(1),46
9,SING OPN LIN 18 3001-3003(1),40


Lower Voltage Result Summary - grouped by Scenario and Contingency 

In [9]:
data_fil_lv = data_low[['Scenario', 'Bus Number', 'Contingency']].drop_duplicates()
pivot_lv = pd.DataFrame(pd.pivot(data_fil_lv, index= ['Scenario','Contingency'], columns = 'Bus Number',values = 'Bus Number').to_records())
pivot_lv['Buses'] = pivot_lv['103'].astype(str).str.cat(pivot_lv[['153', '154', '203', '205', '3006', '3007', '3008']].astype(str), sep=',')
pivot_lv['Buses'] =pivot_lv['Buses'].str.replace(',nan','').str.replace('nan,','')
pivot_low = pivot_lv.drop(columns = list(data_fil_lv['Bus Number'].unique()))

In [10]:
pivot_table_lv = pd.DataFrame(pd.pivot_table(data_fil_lv, index= ['Scenario','Contingency'],values = 'Bus Number', aggfunc='count').to_records())
pivot_table_low = pivot_table_lv.rename(columns = {'Bus Number':'Bus Count'})
df_low = pd.merge(pivot_low, pivot_table_low, on=['Scenario','Contingency'], how='inner').sort_values(by='Bus Count', ascending=False).reset_index(drop=True)
df_low

,Scenario,Contingency,Buses,Bus Count
0,"Solar = 0 MW, Hydro = 400 MW, HLS",SING OPN LIN 6 152-153(1),"103,153,154,203,205,3006,3007,3008",12
1,"Solar = 50 MW, Hydro = 400 MW, LLS",BUS 205,"103,153,154,203,3006,3007,3008",9
2,"Solar = 0 MW, Hydro = 600 MW, LLS",SING OPN LIN 6 152-153(1),"103,153,154,203,205,3006,3007,3008",9
3,"Solar = 100 MW, Hydro = 400 MW, HLS",SING OPN LIN 6 152-153(1),"103,153,154,203,205,3006,3007,3008",9
4,"Solar = 100 MW, Hydro = 600 MW, LLS",SING OPN LIN 9 201-202(1),"103,153,154,203,205,3007,3008",9
...,...,...,...,...
169,"Solar = 0 MW, Hydro = 500 MW, HLS",SING OPN LIN 27 3008-3018(1),154,1
170,"Solar = 100 MW, Hydro = 600 MW, HLS",SING OPN LIN 15 203-205(2),154,1
171,"Solar = 100 MW, Hydro = 600 MW, HLS",SING OPN LIN 14 203-205(1),154,1
172,"Solar = 100 MW, Hydro = 500 MW, HLS",SING OPN LIN 8 153-154(2),154,1
